<a href="https://colab.research.google.com/github/halcdev/qcnn/blob/main/VQE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install qiskit==0.42.0
!pip install qiskit_ibm_runtime

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 30.6 MB/s eta 0:00:00
  Created wheel for qiskit: filename=qiskit-0.42.0-py3-none-any.whl size=12919 sha256=ee903d59584e0448489ff09b5583ac3be4050525f292b7052405682d2412c002
  Stored in directory: /root/.cache/pip/wheels/45/48/9e/b72192ab554955cfe6eaf284324e50cdb0d716427a294ad0dd
Successfully built qiskit
  Attempting uninstall: qiskit-terra
    Found existing installation: qiskit-terra 0.24.0
    Uninstalling qiskit-terra-0.24.0:
      Successfully uninstalled qiskit-terra-0.24.0
  Attempting uninstall: qiskit
    Found existing installation: qiskit 0.43.0
    Uninstalling qiskit-0.43.0:
      Successfully uninstalled qiskit-0.43.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached qiskit_terra-0.24.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (5.9 MB)
  Attempting uninstall: qiskit-terra
    Found existing installation: qiskit-terra 0.23.2
    Uninstalling qiskit-terra-0.23.2:
      Successfully uninstalled qiskit-terra-0.23.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
qiskit 0.42.0 requires qiskit-terra==0.23.2, but you have qiskit-terra 0.24.1 which is incompatible.


In [ ]:
import qiskit
from qiskit.algorithms.optimizers import OptimizerResult
from qiskit.algorithms import VQE

def my_minimizer(fun, x0, jac=None, bounds=None) -> OptimizerResult:
    # Note that the callable *must* have these argument names!
    # Args:
    #     fun (callable): the function to minimize
    #     x0 (np.ndarray): the initial point for the optimization
    #     jac (callable, optional): the gradient of the objective function
    #     bounds (list, optional): a list of tuples specifying the parameter bounds

    result = OptimizerResult()
    # result.x = # optimal parameters
    # result.fun = # optimal function value
    return result



In [ ]:
from qiskit_ibm_runtime import QiskitRuntimeService, Estimator, Session, Options

service = QiskitRuntimeService.save_account(overwrite=True, instance="ibm_quantum",token="0dd239e43f11a659f34407731d29f31c2bf9e65a6329e4a67961c3b02eb2f6e4f2765287ba275b184cb6fa290688992f8dc24ff8afb508de52e6faf4b32376a7")

backend = "ibmq_qasm_simulator"

from qiskit.algorithms.minimum_eigensolvers import VQE

# Use RealAmplitudes circuit to create trial states
from qiskit.circuit.library import RealAmplitudes

ansatz = RealAmplitudes(num_qubits=2, reps=2)

# Search for better states using SPSA algorithm
from qiskit.algorithms.optimizers import SPSA

optimizer = SPSA(150)

# Set a starting point for reproduceability
import numpy as np

np.random.seed(6)
initial_point = np.random.uniform(-np.pi, np.pi, 1)

# Create an object to store intermediate results
from dataclasses import dataclass


@dataclass
class VQELog:
    values: list
    parameters: list

    def update(self, count, parameters, mean, _metadata):
        self.values.append(mean)
        self.parameters.append(parameters)
        print(f"Running circuit {count} of ~350", end="\r", flush=True)


log = VQELog([], [])

In [ ]:
# Main calculation

import qiskit
from qiskit import Aer
from qiskit.circuit.library import TwoLocal
from qiskit.algorithms import VQE
from qiskit.algorithms.optimizers import COBYLA
from qiskit.utils import QuantumInstance
from qiskit.opflow import PauliSumOp
from qiskit.quantum_info.operators import Pauli

token = "0dd239e43f11a659f34407731d29f31c2bf9e65a6329e4a67961c3b02eb2f6e4f2765287ba275b184cb6fa290688992f8dc24ff8afb508de52e6faf4b32376a7"
QiskitRuntimeService.delete_account()
service = QiskitRuntimeService.save_account(channel="ibm_quantum", token=token, overwrite=True)

with Session(service=service, backend=backend) as session:
    options = Options()
    options.optimization_level = 3

    vqe = VQE(
        Estimator(session=session, options=options),
        ansatz,
        optimizer,
        callback=log.update,
    )

    hamiltonian = PauliSumOp.from_list([("I", 1.0), ("Z", 1.0)])

    result = vqe.compute_minimum_eigenvalue(hamiltonian)
    print("Experiment complete.".ljust(30))
    print(f"Raw result: {result.optimal_value}")

    # if "simulator" not in backend:
    #     # Run once with ZNE error mitigation
    #     options.resilience_level = 2
    #     vqe = VQE(
    #         Estimator(session=session, options=options),
    #         ansatz,
    #         SPSA(1),
    #         initial_point=result.optimal_point,
    #     )
    #     result = vqe.compute_minimum_eigenvalue(hamiltonian)
    #     print(f"Mitigated result: {result.optimal_value}")

<ipython-input-37-b7bcb0583c3a>:20: DeprecationWarning: The class ``qiskit.algorithms.minimum_eigen_solvers.vqe.VQE`` is deprecated as of qiskit-terra 0.24.0. It will be removed no earlier than 3 months after the release date. Instead, use the class ``qiskit.algorithms.minimum_eigensolvers.VQE``. See https://qisk.it/algo_migration for a migration guide.
  vqe = VQE(


AttributeError: ignored

In [ ]:
import matplotlib.pyplot as plt

plt.rcParams["font.size"] = 14

# Plot energy and reference value
plt.figure(figsize=(12, 6))
plt.plot(log.values, label="Estimator VQE")
plt.axhline(y=real_solution.groundenergy, color="tab:red", ls="--", label="Target")

plt.legend(loc="best")
plt.xlabel("Iteration")
plt.ylabel("Energy [H]")
plt.title("VQE energy")
plt.show()

In [ ]:
!pip3 install qiskit

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 22.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 41.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.5/241.5 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 64.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.5/37.5 MB 20.7 MB/s eta 0:00:00


In [ ]:
import qiskit
from qiskit import Aer
from qiskit.circuit.library import TwoLocal
from qiskit.algorithms import VQE
from qiskit.algorithms.optimizers import COBYLA
from qiskit.utils import QuantumInstance
from qiskit.opflow import PauliSumOp
from qiskit.quantum_info.operators import Pauli

# Define a simple Hamiltonian in the form of a Pauli Sum (an identity I and a Z Pauli matrix)
H = PauliSumOp.from_list([("I", 1.0), ("Z", 1.0)])

# Define the ansatz (variational form)
ansatz = TwoLocal(rotation_blocks="ry", entanglement_blocks="cz")

# Define the classical optimizer
optimizer = COBYLA()

# Define the backend
backend = Aer.get_backend("statevector_simulator")
quantum_instance = QuantumInstance(backend)

# Run VQE
vqe = VQE(ansatz, optimizer, quantum_instance=quantum_instance)
result = vqe.compute_minimum_eigenvalue(H)

print("Estimated ground state energy: ", result.eigenvalue.real)


<ipython-input-5-fa540e806236>:21: DeprecationWarning: The class ``qiskit.utils.quantum_instance.QuantumInstance`` is deprecated as of qiskit-terra 0.24.0. It will be removed no earlier than 3 months after the release date. For code migration guidelines, visit https://qisk.it/qi_migration.
  quantum_instance = QuantumInstance(backend)
<ipython-input-5-fa540e806236>:24: DeprecationWarning: The class ``qiskit.algorithms.minimum_eigen_solvers.vqe.VQE`` is deprecated as of qiskit-terra 0.24.0. It will be removed no earlier than 3 months after the release date. Instead, use the class ``qiskit.algorithms.minimum_eigensolvers.VQE``. See https://qisk.it/algo_migration for a migration guide.
  vqe = VQE(ansatz, optimizer, quantum_instance=quantum_instance)


Estimated ground state energy:  1.3936730088e-08
